In [ ]:
# train_medium_cnn.py
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pickle
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import time
import json
import warnings
warnings.filterwarnings('ignore')

class MediumConfig:

    DATA_DIRS = {
        'train': './dataset/train',
        'val': './dataset/val',
        'test': './dataset/test'
    }
    
    TRAIN_SAMPLES = 10000    
    VAL_SAMPLES = 4000      
    
    INPUT_DIM = 255          
    OUTPUT_DIM = 50          

    BATCH_SIZE = 128         
    EPOCHS = 150             
    LEARNING_RATE = 0.0005   
    WEIGHT_DECAY = 1e-4      
    DROPOUT_RATE = 0.25      

    OPTIMIZER = 'AdamW'
    USE_GRADIENT_CLIP = False  
    
    LR_SCHEDULER = 'reduce_on_plateau'
    LR_PATIENCE = 8
    LR_FACTOR = 0.5

    EARLY_STOP_PATIENCE = 50  
    EARLY_STOP_MIN_DELTA = 0.0001

    NUM_WORKERS = 0          
    PIN_MEMORY = False
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    SAVE_DIR = Path('./medium_cnn_training')
    SAVE_DIR.mkdir(exist_ok=True)
    
    def __init__(self):
        print("=" * 70)
        print("中等规模训练配置 - 1D CNN架构")
        print("=" * 70)
        print(f"数据规模: {self.TRAIN_SAMPLES:,}训练 + {self.VAL_SAMPLES:,}验证")
        print(f"模型架构: 1D卷积网络 (处理5通道×50点空间信号)")
        print(f"训练参数: LR={self.LEARNING_RATE}, BS={self.BATCH_SIZE}, Dropout={self.DROPOUT_RATE}")
        print(f"最大周期: {self.EPOCHS}, 早停耐心: {self.EARLY_STOP_PATIENCE}")
        print(f"使用设备: {self.DEVICE}")
        if torch.cuda.is_available():
            print(f"GPU: {torch.cuda.get_device_name(0)}")

# ==================== 智能数据加载器 (保持不变) ====================
class SmartBeamDataset(Dataset):
    def __init__(self, data_dir, max_samples=None, shuffle=True):
        self.data_dir = Path(data_dir)
        self.max_samples = max_samples
        self.shuffle = shuffle
        self.samples = []
        
        print(f"\n加载数据从: {data_dir}")
        print(f" 目标样本数: {max_samples if max_samples else '全部'}")
        
        start_time = time.time()
        self._load_and_cache_data()
        load_time = time.time() - start_time
        
        print(f"加载完成: {len(self.samples):,} 个样本")
    
    def _load_and_cache_data(self):
        batch_files = sorted(self.data_dir.glob('batch_*.pkl'))
        
        if not batch_files:
            raise FileNotFoundError(f"未找到批次文件: {self.data_dir}")
        
        print(f"   发现 {len(batch_files)} 个批次文件")
        
        loaded_samples = 0
        for file_idx, batch_file in enumerate(batch_files):
            if self.max_samples and loaded_samples >= self.max_samples:
                break

            with open(batch_file, 'rb') as f:
                batch_data = pickle.load(f)

            remaining = self.max_samples - loaded_samples if self.max_samples else len(batch_data)
            samples_to_add = min(len(batch_data), remaining)

            self.samples.extend(batch_data[:samples_to_add])
            loaded_samples += samples_to_add

            if (file_idx + 1) % 5 == 0 or (file_idx + 1) == len(batch_files):
                print(f"   [{file_idx+1:03d}/{len(batch_files)}] 已加载 {loaded_samples:,} 样本")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        sample = self.samples[idx]

        freq = torch.FloatTensor(sample['input']['frequencies'][:5])
        mode_shapes = torch.FloatTensor(sample['input']['mode_shapes'][:5])
        x = torch.cat([freq, mode_shapes.flatten()])
        y = torch.FloatTensor(sample['output']['binary_labels'])

        metadata = {
            'sample_id': sample.get('sample_id', idx),
            'damage_ratio': np.mean(sample['output']['binary_labels'])
        }
        
        return x, y, metadata

class BeamDamage1DCNN(nn.Module):
    """
    一维卷积网络 for 空间损伤识别。
    设计哲学：将前5阶振型视为5个通道的1D信号，沿梁长进行卷积。
    """
    def __init__(self, n_freq=5, n_points=50, dropout_rate=0.25):
        super().__init__()

        self.conv_block1 = nn.Sequential(
            nn.Conv1d(in_channels=n_freq, out_channels=32, kernel_size=5, padding=2),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )
        self.conv_block2 = nn.Sequential(
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, padding=2),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )
        self.conv_block3 = nn.Sequential(
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=7, dilation=2, padding=6), # dilation=2增大感受野
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )
        

        self.freq_fc = nn.Sequential(
            nn.Linear(n_freq, 16),
            nn.ReLU(),
        )
        

        self.fusion_fc = nn.Sequential(
            nn.Linear(128 + 16, 64), 
            nn.ReLU(),
            nn.Dropout(dropout_rate * 0.5), 
            nn.Linear(64, 1)  
        )
        
    def forward(self, x):

        batch_size = x.shape[0]

        frequencies = x[:, :5]  # (batch, 5)
        mode_shapes = x[:, 5:].view(batch_size, 5, -1)  # (batch, 5, 50)

        spatial_feat = self.conv_block1(mode_shapes)  # -> (batch, 32, 50)
        spatial_feat = self.conv_block2(spatial_feat) # -> (batch, 64, 50)
        spatial_feat = self.conv_block3(spatial_feat) # -> (batch, 128, 50)

        spatial_feat = spatial_feat.transpose(1, 2)

        global_freq_feat = self.freq_fc(frequencies)  # -> (batch, 16)

        global_freq_feat = global_freq_feat.unsqueeze(1).repeat(1, spatial_feat.size(1), 1)
        

        fused_feat = torch.cat([spatial_feat, global_freq_feat], dim=-1)  # (batch, 50, 128+16)

        fused_feat_flat = fused_feat.reshape(-1, fused_feat.size(-1))  # (batch*50, 144)
        logits_flat = self.fusion_fc(fused_feat_flat)  # (batch*50, 1)
        logits = logits_flat.view(batch_size, -1)  # (batch, 50)
        
        return logits

class EarlyStopping:

    def __init__(self, patience=10, min_delta=0.0, verbose=True):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.verbose = verbose
    
    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.verbose:
                print(f'  早停计数器: {self.counter}/{self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0
        return self.early_stop

class TrainingMonitor:

    def __init__(self):
        self.history = {
            'train_loss': [], 'val_loss': [], 'val_acc': [],
            'learning_rate': [], 'epoch_time': [], 'train_acc': []
        }
        self.start_time = time.time()
    
    def update(self, train_loss, val_loss, val_acc, lr, epoch_time, train_acc=None):
        self.history['train_loss'].append(train_loss)
        self.history['val_loss'].append(val_loss)
        self.history['val_acc'].append(val_acc)
        self.history['learning_rate'].append(lr)
        self.history['epoch_time'].append(epoch_time)
        if train_acc is not None:
            self.history['train_acc'].append(train_acc)
    
    def print_epoch_summary(self, epoch, total_epochs, train_loss, val_loss, val_acc, lr, epoch_time):

        elapsed = time.time() - self.start_time
        eta = elapsed / epoch * (total_epochs - epoch) if epoch > 0 else 0
        
        print(f'  Epoch {epoch:3d}/{total_epochs} | '
              f'时间: {epoch_time:5.1f}s | '
              f'LR: {lr:.6f} | '
              f'训练损失: {train_loss:.4f} | '
              f'验证损失: {val_loss:.4f} | '
              f'验证准确率: {val_acc:.3f} | '
              f'总耗时: {elapsed/60:.1f}m | '
              f'ETA: {eta/60:.1f}m')

def compute_accuracy(predictions, targets, threshold=0.5):

    probs = torch.sigmoid(predictions)
    binary_preds = (probs > threshold).float()
    correct = (binary_preds == targets).float()
    return correct.mean().item()

def train_epoch(model, loader, criterion, optimizer, device, config):

    model.train()
    total_loss = 0.0
    total_acc = 0.0
    total_samples = 0
    
    for batch_idx, (batch_x, batch_y, _) in enumerate(loader):
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        batch_size = batch_x.size(0)

        optimizer.zero_grad()
        predictions = model(batch_x)  
        loss = criterion(predictions, batch_y)

        loss.backward()

        if config.USE_GRADIENT_CLIP:
            torch.nn.utils.clip_grad_norm_(model.parameters(), config.GRADIENT_CLIP_VAL)
        
        optimizer.step()

        total_loss += loss.item() * batch_size
        total_acc += compute_accuracy(predictions, batch_y) * batch_size
        total_samples += batch_size

        if (batch_idx + 1) % 20 == 0:
            avg_loss = total_loss / total_samples
            avg_acc = total_acc / total_samples
            print(f'    批次 {batch_idx+1:03d}/{len(loader)} | '
                  f'损失: {avg_loss:.4f} | '
                  f'准确率: {avg_acc:.3f}')
    
    avg_loss = total_loss / total_samples
    avg_acc = total_acc / total_samples
    return avg_loss, avg_acc

def validate(model, loader, criterion, device):

    model.eval()
    total_loss = 0.0
    total_acc = 0.0
    total_samples = 0
    
    all_preds, all_targets = [], []
    
    with torch.no_grad():
        for batch_x, batch_y, _ in loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            batch_size = batch_x.size(0)
            
            predictions = model(batch_x)  
            loss = criterion(predictions, batch_y)
            
            total_loss += loss.item() * batch_size
            total_acc += compute_accuracy(predictions, batch_y) * batch_size
            total_samples += batch_size
            
            all_preds.append(predictions.cpu())
            all_targets.append(batch_y.cpu())
    
    avg_loss = total_loss / total_samples
    avg_acc = total_acc / total_samples

    all_preds = torch.cat(all_preds, dim=0)
    all_targets = torch.cat(all_targets, dim=0)
    
    return avg_loss, avg_acc, all_preds, all_targets

def main():
    config = MediumConfig()

    print("\n" + "=" * 70)
    print("阶段1: 加载数据集")
    print("=" * 70)
    
    train_dataset = SmartBeamDataset(
        config.DATA_DIRS['train'], 
        max_samples=config.TRAIN_SAMPLES,
        shuffle=True
    )
    
    val_dataset = SmartBeamDataset(
        config.DATA_DIRS['val'], 
        max_samples=config.VAL_SAMPLES,
        shuffle=False
    )
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=config.BATCH_SIZE,
        shuffle=True,
        num_workers=config.NUM_WORKERS,
        pin_memory=config.PIN_MEMORY
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=config.BATCH_SIZE,
        shuffle=False,
        num_workers=config.NUM_WORKERS,
        pin_memory=config.PIN_MEMORY
    )
    
    print(f"\n数据统计:")
    print(f"   训练集: {len(train_dataset):,} 样本, {len(train_loader)} 批次")
    print(f"   验证集: {len(val_dataset):,} 样本, {len(val_loader)} 批次")

    print("\n" + "=" * 70)
    print("阶段2: 初始化1D CNN模型")
    print("=" * 70)
    
    model = BeamDamage1DCNN(
        n_freq=5,
        n_points=50,
        dropout_rate=config.DROPOUT_RATE
    ).to(config.DEVICE)
    
    # 计算参数量
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f" 模型参数统计:")
    print(f"   总参数: {total_params:,}")
    print(f"   可训练参数: {trainable_params:,}")
    print(f"   模型大小: {total_params * 4 / 1024**2:.2f} MB (float32)")
    print(f"   CNN架构: 5→[32→64→128]→融合→输出")
    
    pos_weight = torch.tensor([5.0]).to(config.DEVICE)  
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    
    if config.OPTIMIZER == 'AdamW':
        optimizer = optim.AdamW(
            model.parameters(), 
            lr=config.LEARNING_RATE, 
            weight_decay=config.WEIGHT_DECAY
        )
    else:
        optimizer = optim.Adam(
            model.parameters(), 
            lr=config.LEARNING_RATE
        )

    if config.LR_SCHEDULER == 'cosine':
        scheduler = optim.lr_scheduler.CosineAnnealingLR(
            optimizer, T_max=config.EPOCHS, eta_min=1e-6)
    else:
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=config.LR_FACTOR, 
            patience=config.LR_PATIENCE, verbose=True)

    early_stopping = EarlyStopping(
        patience=config.EARLY_STOP_PATIENCE,
        min_delta=config.EARLY_STOP_MIN_DELTA,
        verbose=True
    )

    monitor = TrainingMonitor()
    best_val_acc = 0.0
    best_epoch = 0

    print("\n" + "=" * 70)
    print("阶段3: 开始CNN训练")
    print("=" * 70)
    
    for epoch in range(1, config.EPOCHS + 1):
        epoch_start = time.time()

        train_loss, train_acc = train_epoch(
            model, train_loader, criterion, optimizer, config.DEVICE, config
        )

        val_loss, val_acc, val_preds, val_targets = validate(
            model, val_loader, criterion, config.DEVICE
        )

        if config.LR_SCHEDULER == 'reduce_on_plateau':
            scheduler.step(val_loss)
        else:
            scheduler.step()
        
        current_lr = optimizer.param_groups[0]['lr']
        epoch_time = time.time() - epoch_start

        monitor.update(train_loss, val_loss, val_acc, current_lr, epoch_time, train_acc)

        monitor.print_epoch_summary(
            epoch, config.EPOCHS, train_loss, val_loss, val_acc, 
            current_lr, epoch_time
        )

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_epoch = epoch
            
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'train_acc': train_acc,
                'val_loss': val_loss,
                'val_acc': val_acc,
                'config': config.__dict__,
                'history': monitor.history
            }, config.SAVE_DIR / 'best_model.pth')
            
            print(f"  保存最佳模型 @ Epoch {epoch}: 准确率 = {val_acc:.3f}")

        if epoch % 10 == 0:
            checkpoint_path = config.SAVE_DIR / f'checkpoint_epoch_{epoch}.pth'
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_acc': val_acc,
            }, checkpoint_path)
            print(f"  检查点已保存: {checkpoint_path}")

        if early_stopping(val_loss):
            print(f"\n  早停触发于第 {epoch} 周期")
            print(f"   最佳验证准确率: {best_val_acc:.3%} @ Epoch {best_epoch}")

    print("\n" + "=" * 70)
    print("训练完成!")
    print("=" * 70)

    final_model_path = config.SAVE_DIR / 'final_model.pth'
    torch.save(model.state_dict(), final_model_path)

    history_path = config.SAVE_DIR / 'training_history.npy'
    np.save(history_path, monitor.history)

    config_dict = {k: v for k, v in config.__dict__.items() 
                  if not k.startswith('_') and not callable(v)}
    
    with open(config.SAVE_DIR / 'training_config.json', 'w') as f:
        json.dump(config_dict, f, indent=2, default=str)
    
    print(f"\n 训练总结:")
    print(f"   总训练周期: {len(monitor.history['train_loss'])}")
    print(f"   最佳验证准确率: {best_val_acc:.3%} @ Epoch {best_epoch}")
    print(f"   最终验证准确率: {monitor.history['val_acc'][-1]:.3%}")
    print(f"   训练/验证损失比: {monitor.history['train_loss'][-1]/monitor.history['val_loss'][-1]:.2f}")

    print(f"\n 保存的文件:")
    print(f"   最佳模型: {config.SAVE_DIR / 'best_model.pth'}")
    print(f"   最终模型: {final_model_path}")
    print(f"   训练历史: {history_path}")
    print(f"   训练配置: {config.SAVE_DIR / 'training_config.json'}")

    plot_training_curves(monitor.history, config.SAVE_DIR)
    
    return model, monitor.history, best_val_acc

def plot_training_curves(history, save_dir):

    epochs = range(1, len(history['train_loss']) + 1)
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle('1D CNN Training Results (10k Train + 4k Val)', fontsize=16, y=1.02)

    ax = axes[0, 0]
    ax.plot(epochs, history['train_loss'], 'b-', linewidth=2, label='Training Loss')
    ax.plot(epochs, history['val_loss'], 'r-', linewidth=2, label='Validation Loss')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
    ax.set_title('Training & Validation Loss')
    ax.legend()
    ax.grid(True, alpha=0.3)
    if max(history['train_loss']) > 10 * max(history['val_loss']):
        ax.set_yscale('log')

    ax = axes[0, 1]
    ax.plot(epochs, history['val_acc'], 'g-', linewidth=2, label='Validation Acc')
    if 'train_acc' in history and len(history['train_acc']) == len(epochs):
        ax.plot(epochs, history['train_acc'], 'b--', linewidth=1.5, alpha=0.7, label='Training Acc')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Accuracy')
    ax.set_title('Training & Validation Accuracy')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_ylim([0.5, 1.0])

    ax = axes[0, 2]
    ax.plot(epochs, history['learning_rate'], 'm-', linewidth=2)
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Learning Rate')
    ax.set_title('Learning Rate Schedule')
    ax.grid(True, alpha=0.3)
    ax.set_yscale('log')

    ax = axes[1, 0]
    ax.plot(epochs, history['epoch_time'], 'c-', linewidth=2)
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Time (seconds)')
    ax.set_title('Epoch Duration')
    ax.grid(True, alpha=0.3)
    ax.axhline(y=np.mean(history['epoch_time']), color='r', linestyle='--', alpha=0.5, 
               label=f'Mean: {np.mean(history["epoch_time"]):.1f}s')
    ax.legend()

    ax = axes[1, 1]
    scatter = ax.scatter(history['val_loss'], history['val_acc'], 
                        c=range(len(history['val_loss'])), cmap='viridis', 
                        s=30, alpha=0.7)
    ax.set_xlabel('Validation Loss')
    ax.set_ylabel('Validation Accuracy')
    ax.set_title('Loss vs Accuracy Correlation')
    ax.grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=ax, label='Epoch')

    ax = axes[1, 2]
    early_epochs = min(30, len(epochs))
    ax.plot(epochs[:early_epochs], history['train_loss'][:early_epochs], 'b-o', 
            linewidth=1.5, markersize=3, label='Training Loss')
    ax.plot(epochs[:early_epochs], history['val_loss'][:early_epochs], 'r-s', 
            linewidth=1.5, markersize=3, label='Validation Loss')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
    ax.set_title(f'Early Training (First {early_epochs} Epochs)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plot_path = save_dir / 'training_curves.png'
    plt.savefig(plot_path, dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"\n 训练曲线已保存: {plot_path}")

def quick_system_check():
    """快速系统检查"""
    print(" 系统检查开始...")
    
    checks_passed = True

    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f" GPU: {gpu_name} ({gpu_memory:.1f} GB)")
    else:
        print("  未检测到GPU，将使用CPU训练（会很慢）")

    data_dirs = ['./dataset/train', './dataset/val', './dataset/test']
    for dir_path in data_dirs:
        path = Path(dir_path)
        if path.exists():
            pkl_files = list(path.glob('batch_*.pkl'))
            print(f" {dir_path}: 发现 {len(pkl_files)} 个批次文件")
        else:
            print(f" {dir_path}: 目录不存在")
            checks_passed = False
    
    return checks_passed

if __name__ == "__main__":
    print("=" * 70)
    print("梁结构损伤识别 - 1D CNN中等规模训练")
    print("=" * 70)

    if not quick_system_check():
        print("\n系统检查失败，请检查以上问题")
        exit(1)
    
    print("\n系统检查通过，开始训练流程")
    
    try:
 
        model, history, best_acc = main()
        
        print("\n" + "=" * 70)
        print("🎉 CNN训练成功完成!")
        print("=" * 70)

        original_fcn_acc = 0.82  # 原始FCN性能
        improvement = best_acc - original_fcn_acc
        print(f"\n性能对比:")
        print(f"   原始FCN (10k数据): {original_fcn_acc:.1%}")
        print(f"   1D CNN (10k数据): {best_acc:.1%}")
        if improvement > 0:
            print(f"   性能提升: +{improvement*100:.1f}%")
        else:
            print(f"   性能变化: {improvement*100:.1f}%")
        
    except Exception as e:
        print(f"\n训练过程中出错: {e}")
        import traceback
        traceback.print_exc()